# Loop 87 Analysis: Strategic Assessment

## Key Questions:
1. What is the CV-LB relationship?
2. What approaches have been tried?
3. What might change the CV-LB relationship?
4. What should we try next?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Submission data with known LB scores
submissions = [
    ('exp_000', 0.011081, 0.09816),
    ('exp_001', 0.012297, 0.10649),
    ('exp_003', 0.010501, 0.09719),
    ('exp_005', 0.01043, 0.09691),
    ('exp_006', 0.009749, 0.09457),
    ('exp_007', 0.009262, 0.09316),
    ('exp_009', 0.009192, 0.09364),
    ('exp_012', 0.009004, 0.09134),
    ('exp_024', 0.008689, 0.08929),
    ('exp_026', 0.008465, 0.08875),
    ('exp_030', 0.008298, 0.08772),
    ('exp_035', 0.009825, 0.09696),
    ('exp_067', 0.008303, 0.08774),
]

df = pd.DataFrame(submissions, columns=['exp', 'cv', 'lb'])
print("Submissions with known LB scores:")
print(df.to_string(index=False))

In [ ]:
# Fit linear regression: LB = slope * CV + intercept
X = df['cv'].values.reshape(-1, 1)
y = df['lb'].values

reg = LinearRegression()
reg.fit(X, y)

slope = reg.coef_[0]
intercept = reg.intercept_
r2 = reg.score(X, y)

print(f"\n=== CV-LB RELATIONSHIP ===")
print(f"Linear fit: LB = {slope:.4f} * CV + {intercept:.4f}")
print(f"R² = {r2:.4f}")
print(f"\nInterpretation:")
print(f"  - Slope: {slope:.2f}x (CV improvement translates to {slope:.2f}x LB improvement)")
print(f"  - Intercept: {intercept:.4f} (structural gap even at CV=0)")

# Target analysis
target = 0.0347
print(f"\n=== TARGET ANALYSIS ===")
print(f"Target LB: {target}")
print(f"Intercept: {intercept:.4f}")
print(f"Gap: intercept - target = {intercept - target:.4f}")

if intercept > target:
    required_cv = (target - intercept) / slope
    print(f"\n⚠️ CRITICAL: Intercept ({intercept:.4f}) > Target ({target})")
    print(f"Required CV to hit target: {required_cv:.6f} (IMPOSSIBLE - negative!)")
else:
    required_cv = (target - intercept) / slope
    print(f"Required CV to hit target: {required_cv:.6f}")

In [ ]:
# Visualize the CV-LB relationship
plt.figure(figsize=(10, 6))

# Plot submissions
plt.scatter(df['cv'], df['lb'], s=100, c='blue', alpha=0.7, label='Submissions')

# Plot regression line
cv_range = np.linspace(0, 0.015, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f}')

# Plot target
plt.axhline(y=target, color='green', linestyle='-', linewidth=2, label=f'Target: {target}')

# Plot intercept
plt.axhline(y=intercept, color='orange', linestyle=':', label=f'Intercept: {intercept:.4f}')

# Annotate best submissions
best_idx = df['lb'].idxmin()
plt.annotate(f"Best: {df.loc[best_idx, 'exp']}\nCV={df.loc[best_idx, 'cv']:.4f}\nLB={df.loc[best_idx, 'lb']:.4f}",
             xy=(df.loc[best_idx, 'cv'], df.loc[best_idx, 'lb']),
             xytext=(df.loc[best_idx, 'cv']+0.001, df.loc[best_idx, 'lb']+0.01),
             arrowprops=dict(arrowstyle='->', color='black'))

plt.xlabel('CV Score (MSE)')
plt.ylabel('LB Score (MSE)')
plt.title('CV vs LB Relationship - All Submissions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship_loop87.png', dpi=100)
plt.show()

print("\nPlot saved to /home/code/exploration/cv_lb_relationship_loop87.png")

In [ ]:
# Analyze the gap
print("=" * 60)
print("CRITICAL ANALYSIS: THE INTERCEPT PROBLEM")
print("=" * 60)

print(f"""
The CV-LB relationship shows:
- LB ≈ {slope:.2f} * CV + {intercept:.4f}
- R² = {r2:.4f} (very strong linear relationship)

The intercept ({intercept:.4f}) represents the STRUCTURAL DISTRIBUTION SHIFT:
- Even if we achieved CV = 0 (perfect training fit), LB would still be ~{intercept:.4f}
- This is because test solvents are fundamentally different from training solvents
- The intercept ({intercept:.4f}) > target ({target}) means:
  → NO AMOUNT OF CV OPTIMIZATION CAN REACH THE TARGET
  → We need approaches that REDUCE THE INTERCEPT, not just improve CV

What has been tried (all fall on the same line):
- MLP variants (exp_000, exp_003, exp_005, exp_006, exp_007)
- LightGBM (exp_001)
- CatBoost + XGBoost (exp_049, exp_050)
- GP ensemble (exp_030)
- Extrapolation detection (exp_068, exp_070, exp_071)
- Similarity weighting (exp_082)
- Pseudo-labeling (exp_083)
- Solvent clustering (exp_081)

ALL approaches fall on the same CV-LB line!
""")

In [ ]:
# What would it take to reach the target?
print("=" * 60)
print("WHAT WOULD IT TAKE TO REACH THE TARGET?")
print("=" * 60)

print(f"""
Option 1: Reduce the intercept
- Current intercept: {intercept:.4f}
- Target: {target}
- Need to reduce intercept by: {intercept - target:.4f}
- This requires fundamentally different approaches that generalize better to unseen solvents

Option 2: Change the slope
- Current slope: {slope:.2f}
- If slope were 1.0 (perfect CV-LB correlation), we'd need CV = {target - intercept:.4f} (still impossible)
- Slope change alone won't help if intercept > target

Option 3: Find approaches that don't follow the same CV-LB line
- The top leaderboard score is 0.03470 (exactly at target!)
- 2nd place is 0.07074 (2x worse than 1st)
- This HUGE gap suggests 1st place found a fundamentally different approach
- They likely have a DIFFERENT CV-LB relationship (lower intercept)

KEY INSIGHT:
- The 1st place team achieved LB = 0.0347
- If they followed our CV-LB line: CV = (0.0347 - {intercept:.4f}) / {slope:.2f} = {(0.0347 - intercept) / slope:.4f} (IMPOSSIBLE)
- Therefore, they MUST have a different CV-LB relationship
- Their intercept is likely much lower (< 0.0347)
""")

In [ ]:
# What approaches might change the CV-LB relationship?
print("=" * 60)
print("APPROACHES THAT MIGHT CHANGE THE CV-LB RELATIONSHIP")
print("=" * 60)

print("""
1. GRAPH NEURAL NETWORKS (GNN)
   - The GNN benchmark achieved 0.0039 CV
   - GNNs can learn molecular structure directly
   - May generalize better to unseen solvents through learned molecular representations
   - TRIED: exp_040 (failed due to implementation issues)
   - SHOULD TRY: Proper GNN implementation with message passing

2. TRANSFER LEARNING / PRE-TRAINING
   - Pre-train on large molecular datasets (e.g., QM9, PubChem)
   - Fine-tune on catechol data
   - The pre-trained representations may generalize better
   - NOT TRIED YET

3. PHYSICS-INFORMED CONSTRAINTS
   - Arrhenius kinetics (already used, helps CV but not intercept)
   - Solvent polarity constraints
   - Reaction mechanism constraints
   - PARTIALLY TRIED: Arrhenius features help CV but don't reduce intercept

4. DOMAIN ADAPTATION
   - Adversarial training to make representations domain-invariant
   - Test-time adaptation
   - TRIED: Pseudo-labeling (exp_083) - didn't help

5. ENSEMBLE OF FUNDAMENTALLY DIFFERENT APPROACHES
   - Combine GNN + MLP + GP
   - Each may have different CV-LB relationships
   - Ensemble might have lower intercept
   - NOT TRIED YET

6. STUDY TOP PUBLIC KERNELS MORE CAREFULLY
   - The ens-model kernel uses official LOO-CV but achieves good LB
   - What's different? Feature engineering? Ensemble weighting?
   - TRIED: exp_080 (ens_model) - CV 0.010266, worse than our best
   - Need to understand what we're missing
""")

In [ ]:
# Check what the best experiments look like
print("=" * 60)
print("BEST EXPERIMENTS ANALYSIS")
print("=" * 60)

best_cv_exps = [
    ('exp_049', 0.008092, 'CatBoost + XGBoost ensemble', 'pending'),
    ('exp_050', 0.008092, 'CatBoost + XGBoost fixed', 'pending'),
    ('exp_030', 0.008298, 'GP ensemble', 0.08772),
    ('exp_067', 0.008303, 'Revert to exp_030', 0.08774),
    ('exp_026', 0.008465, 'Weighted loss', 0.08875),
]

print("Top 5 experiments by CV:")
for exp, cv, desc, lb in best_cv_exps:
    lb_str = f"{lb:.5f}" if isinstance(lb, float) else lb
    print(f"  {exp}: CV={cv:.6f}, LB={lb_str} - {desc}")

print(f"""

KEY OBSERVATIONS:
1. exp_049/exp_050 have the best CV (0.008092) but LB is pending
2. exp_030 has CV=0.008298 and LB=0.08772 (our best verified LB)
3. The CV improvement from 0.008298 to 0.008092 is only 2.5%
4. Based on the CV-LB relationship, this would translate to:
   - Predicted LB for exp_049: {slope:.2f} * 0.008092 + {intercept:.4f} = {slope * 0.008092 + intercept:.4f}
   - This is only slightly better than exp_030's LB (0.08772)

CONCLUSION:
- Submitting exp_049 might give us a slightly better LB (~0.0869)
- But this is still FAR from the target (0.0347)
- We need a fundamentally different approach to reduce the intercept
""")

In [ ]:
# Strategic recommendations
print("=" * 60)
print("STRATEGIC RECOMMENDATIONS FOR LOOP 87")
print("=" * 60)

print("""
Given:
- 4 submissions remaining
- Best CV: 0.008092 (exp_049/exp_050) - not yet submitted
- Best LB: 0.0877 (exp_030/exp_067)
- Target: 0.0347
- Intercept problem: 0.052 > 0.0347

OPTION A: Submit exp_049 to verify CV-LB relationship
- Pros: Might get slightly better LB (~0.0869)
- Cons: Uses 1 of 4 remaining submissions, won't reach target
- Verdict: WORTH DOING to verify the CV-LB relationship holds

OPTION B: Try fundamentally different approaches
- GNN with proper implementation
- Transfer learning from pre-trained molecular models
- Physics-informed neural networks
- Pros: Might change the CV-LB relationship
- Cons: High risk, may not work
- Verdict: MUST TRY - only way to reach target

OPTION C: Study top kernels more carefully
- The ens-model kernel uses official LOO-CV
- What features/techniques are we missing?
- Pros: Proven to work on this competition
- Cons: May not be enough to reach target
- Verdict: WORTH INVESTIGATING

RECOMMENDED STRATEGY:
1. Submit exp_049 to verify CV-LB relationship (use 1 submission)
2. Implement proper GNN with message passing
3. Try transfer learning from pre-trained molecular models
4. Save 2-3 submissions for promising new approaches
""")

In [ ]:
# Final summary
print("=" * 60)
print("LOOP 87 SUMMARY")
print("=" * 60)

print(f"""
CV-LB Relationship: LB = {slope:.2f} * CV + {intercept:.4f} (R² = {r2:.4f})

CRITICAL FINDING:
- Intercept ({intercept:.4f}) > Target ({target})
- ALL 86+ experiments fall on the same CV-LB line
- Standard CV optimization CANNOT reach the target
- We need approaches that REDUCE THE INTERCEPT

BEST RESULTS:
- Best CV: 0.008092 (exp_049/exp_050) - not submitted
- Best LB: 0.0877 (exp_030/exp_067)
- Target: 0.0347
- Gap: 0.0877 - 0.0347 = 0.053 (152.8% above target)

WHAT'S BEEN TRIED (all fall on same line):
- MLP variants, LightGBM, CatBoost, XGBoost, GP
- Various feature sets (Spange, DRFP, combined)
- Extrapolation detection, similarity weighting, pseudo-labeling
- Solvent clustering, conservative blending

WHAT HASN'T BEEN TRIED (might change the line):
- Proper GNN implementation with message passing
- Transfer learning from pre-trained molecular models
- Physics-informed neural networks
- Test-time adaptation

NEXT STEPS:
1. Submit exp_049 to verify CV-LB relationship
2. Implement fundamentally different approaches (GNN, transfer learning)
3. Save submissions for promising new approaches
""")

In [ ]:
# Record finding
finding = f"""Loop 87 Analysis: CV-LB relationship is LB = {slope:.2f}*CV + {intercept:.4f} (R²={r2:.4f}). 
CRITICAL: Intercept ({intercept:.4f}) > Target ({target}). Required CV to hit target is {(target - intercept) / slope:.4f} (IMPOSSIBLE). 
Best LB: 0.0877 (exp_030, exp_067). Best CV: 0.008092 (exp_049/exp_050, not submitted). 
All 86+ experiments fall on the same CV-LB line. Standard CV optimization CANNOT reach target. 
Need approaches that REDUCE THE INTERCEPT (GNN, transfer learning, physics-informed models)."""

print("Finding to record:")
print(finding)